# 3. Training Pipeline

## 3.1. Setup

In [1]:
# Standard imports
import os
import sys
import json
import time
from datetime import date, datetime, timedelta
import warnings
from pathlib import Path
warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

#  Load settings 
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Project root dir: c:\Users\krist\Documents\GitHub\pm25
HopsworksSettings initialized!
2026-01-15 14:41:30,792 INFO: Initializing external client
2026-01-15 14:41:30,794 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-15 14:41:32,428 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


Repository

In [2]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Repository exists at c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn


## 3.2. Get secrets

In [3]:
today = date.today()

if settings.AQICN_API_KEY is None:
    print("AQICN_API_KEY missing.")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
except Exception:
    pass

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [4]:
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

today = datetime.today().date()

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather",
    version=1,
)

## 3.3. Get Feature Groups

In [5]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 3.4. Load Metadata from Feature Group

In [6]:
# Load metadata from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

# Extract unique sensor metadata
metadata_df = aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country"]].drop_duplicates(subset=["sensor_id"])
print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
metadata_df = metadata_df.set_index("sensor_id")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (18.80s) 
📍 Loaded metadata for 103 sensors


## 3.5. Feature View Creation
Create multiple feature views with different feature combinations (baseline, rolling windows, lagged features, nearby sensors, complete) for model comparison.

In [7]:
# from hsfs.feature_view import FeatureView

# fs_id = fs.id

# fv_names = [
#     "air_quality_baseline_fv",
#     "air_quality_rolling_fv",
#     "air_quality_nearby_fv",
#     "air_quality_lagged_1d_fv",
#     "air_quality_lagged_2d_fv",
#     "air_quality_lagged_3d_fv",
#     "air_quality_complete_fv",
# ]

# for name in fv_names:
#     for version in range(1, 10):  # delete versions 1–9
#         try:
#             FeatureView.clean(fs_id, name, version)
#             print(f"Deleted {name} v{version}")
#         except Exception as e:
#             if "wasn't found" in str(e):
#                 continue
#             else:
#                 print(f"Unexpected error cleaning {name} v{version}: {e}")

In [8]:
# Select features for training data
baseline_features = air_quality_fg.select(["pm25", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])

baseline_feature_view = fs.get_or_create_feature_view(
    name="air_quality_baseline_fv",
    description="Weather features for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=baseline_features,
)

rolling_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
rolling_feature_view = fs.get_or_create_feature_view(
    name="air_quality_rolling_fv",
    description="Weather features, PM2.5 rolling window (3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=rolling_features,
)

nearby_features = air_quality_fg.select(["pm25", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
nearby_feature_view = fs.get_or_create_feature_view(
    name="air_quality_nearby_fv",
    description="Weather features, PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=nearby_features,
)

lagged_1d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_1d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_1d_fv",
    description="Weather features, PM2.5 lags (1d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_1d_features,
)

lagged_2d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_2d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_2d_fv",
    description="Weather features, PM2.5 lags (1d, 2d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_2d_features,
)

lagged_3d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_3d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_3d_fv",
    description="Weather features, PM2.5 lags (1d, 2d, 3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_3d_features,
)

complete_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
complete_feature_view = fs.get_or_create_feature_view(
    name="air_quality_complete_fv",
    description="Weather features, PM2.5 rolling window (3d), and PM2.5 lags (1d, 2d, 3d), and PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=complete_features,
)

2026-01-15 14:41:58,736 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-15 14:42:01,130 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-15 14:42:03,074 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-15 14:42:04,984 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-15 14:42

## 3.6. Model Training Setup
Set up test data split date, initialize containers for models and predictions, and define feature view dictionary for iteration

In [9]:
models = defaultdict(dict)
y_preds = defaultdict(dict)
results = []

feature_views = {
    "baseline": baseline_feature_view,
    "rolling": rolling_feature_view,
    "nearby": nearby_feature_view,
    "lagged_1d": lagged_1d_feature_view,
    "lagged_2d": lagged_2d_feature_view,
    "lagged_3d": lagged_3d_feature_view,
    "complete": complete_feature_view,
}

## 3.7. Model Training Loop
Train XGBoost models for each feature combination and sensor, run 5 iterations per configuration, select best model based on R2 score, and store results.

In [10]:
# Should all of these be saved to model registry instead???
model_dir = f"{root_dir}/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [11]:
# calculate the mean for weather features for each day and sensor_id and replace in fg
weather_daily_mean = (
    weather_fg.read()
    .groupby(["sensor_id", "date"])
    .mean()
    .reset_index()
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.15s) 


## 3.8. Debug data quality
To store trained models

In [12]:
# #  DEBUG:
# print("🔍 DEBUGGING DATA QUALITY")
# print("="*60)

# # Check one sensor's data
# first_feature_name = list(feature_views.keys())[0]
# sample_data = feature_views[first_feature_name].query.read()
# test_sensor_id = sample_data["sensor_id"].iloc[0]

# print(f"\n📊 Checking sensor {test_sensor_id}:")

# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     sensor_data = data[data["sensor_id"] == test_sensor_id].copy()
    
#     if len(sensor_data) == 0:
#         print(f"  {feature_name}: NO DATA")
#         continue
    
#     print(f"\n  {feature_name}: {len(sensor_data)} rows")
#     print(f"    Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
#     # Check each column for NaN
#     for col in sensor_data.columns:
#         nan_count = sensor_data[col].isna().sum()
#         nan_pct = (nan_count / len(sensor_data)) * 100
#         if nan_pct > 0:
#             print(f"    {col}: {nan_count}/{len(sensor_data)} NaN ({nan_pct:.1f}%)")
    
#     # Show first few rows
#     print(f"\n    Sample data:")
#     print(sensor_data.head())
#     break  # Only check baseline for now

# print("\n" + "="*60)

Training loop

In [13]:
# print(feature_name, data.shape)
# print(data["sensor_id"].unique()[:10])

In [14]:
# print("=== FEATURE VIEW CHECK ===")
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     print(feature_name, data.shape)
#     print("unique sensors:", len(data["sensor_id"].unique()))
#     print("sample sensors:", data["sensor_id"].unique()[:5])
#     print()

In [15]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in data["sensor_id"].unique():
#         df = data[data["sensor_id"] == sensor_id].copy()
#         print(df.isna().sum())

In [16]:
models = {name: {} for name in feature_views.keys()}
y_preds = {name: {} for name in feature_views.keys()}
results = []

In [17]:
for feature_name, feature_view in feature_views.items():
    data = feature_view.query.read()
    data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

    for sensor_id in data["sensor_id"].unique():

    # for sensor_id in metadata_df.index:
        df = data[data["sensor_id"] == sensor_id].copy()

        # Only drop rows where pm25 (target) is NaN
        df = df.dropna(subset=["pm25"])
        
        if len(df) < 10:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient data ({len(df)} rows)")
            continue

        # Identify which features this feature view actually uses
        # Drop non-feature columns first
        feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "city", "street", "country", "latitude", "longitude", "aqicn_url"]]
        
        # Keep rows with NaN in features - XGBoost handles NaN natively
        df_clean = df.copy()
        
        if len(df_clean) < 10:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient clean data ({len(df_clean)} rows, needed features: {feature_cols})")
            continue

        # Split
        train_size = int(0.8 * len(df_clean))
        train_df = df_clean.iloc[:train_size]
        test_df = df_clean.iloc[train_size:]

        if len(test_df) < 2:
            print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): test set too small ({len(test_df)} rows)")
            continue

        # Prepare features and target
        X_train = train_df[feature_cols]
        y_train = train_df["pm25"]
        X_test = test_df[feature_cols]
        y_test = test_df["pm25"]

        best_r2 = -float('inf')
        best_mse = float('inf')
        best_model = None

        mse_list = []
        r2_list = []
        
        for i in range(5):
            model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            r2_list.append(r2)
            mse_list.append(mse)
            
            if r2 > best_r2:
                best_r2 = r2
                best_mse = mse
                best_model = model

        models[feature_name][sensor_id] = best_model

        if best_model is not None:
            y_preds[feature_name][sensor_id] = best_model.predict(X_test)
            results.append({
                "feature_name": feature_name,
                "sensor_id": sensor_id,
                "MSE": sum(mse_list) / len(mse_list),
                "R2": sum(r2_list) / len(r2_list),
                "train_size": len(X_train),
                "test_size": len(X_test),
            })
            print(f"✅ {feature_name} - Sensor {sensor_id}: R2={best_r2:.3f}, MSE={best_mse:.2f}")
        else:
            print(f"⚠️  No valid model for {feature_name} - {sensor_id}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.72s) 
2026-01-15 14:42:24,201 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-15 14:42:24,202 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-15 14:42:24,206 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-15 14:42:24,207 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-15 14:42:24,283 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-15 14:42:24,284 WARNING: DeprecationWarning: is_sparse i

In [18]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in metadata_df.index:
#         df = data[data["sensor_id"] == sensor_id].copy()

#         # Clean the data before splitting
#         df = df.dropna(subset=["pm25"])
#         # features_for_cleaning = df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])

#         target_for_cleaning = df["pm25"]
#         clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#         df_clean = df[clean_mask].copy()
#         if len(df_clean) < 10:
#             print(f"⚠️  Skipping sensor {sensor_id}: insufficient data after cleaning ({len(df_clean)} rows)")
#             continue

#         train_size = int(0.8 * len(df_clean))
#         train_df = df_clean.iloc[:train_size]
#         test_df = df_clean.iloc[train_size:]

#         if len(test_df) < 2:
#             print(f"⚠️  Skipping sensor {sensor_id}: test set too small after split ({len(test_df)} rows)")
#             continue

#         # Drop non-feature columns (pm25 is target, others are metadata)
#         X_train = train_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_train = train_df["pm25"]
#         X_test = test_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_test = test_df["pm25"]

#         best_r2 = -float('inf')
#         best_mse = float('inf')
#         best_model = None

#         mse_list = []
#         r2_list = []
#         for i in range(5):
#             model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)
#             r2 = r2_score(y_test, y_pred)
#             mse = mean_squared_error(y_test, y_pred)
#             r2_list.append(r2)
#             mse_list.append(mse)
#             if r2 > best_r2:
#                 best_r2 = r2
#                 best_mse = mse
#                 best_model = model

#         models[feature_name][sensor_id] = best_model

#         if best_model is not None:
#             y_preds[feature_name][sensor_id] = best_model.predict(X_test)
#             results.append({
#                 "feature_name": feature_name,
#                 "sensor_id": sensor_id,
#                 "MSE": sum(mse_list) / len(mse_list),
#                 "R2": sum(r2_list) / len(r2_list),
#                 "train_size": len(X_train),
#                 "test_size": len(X_test),
#             })
#         else:
#             print(f"⚠️  No valid model trained for {feature_name} - {sensor_id}, R2 scores: {r2_list}, Best R2: {best_r2}")
            

## 3.9. Model Selection and Saving
Identify best performing model for each sensor, save models and feature importance plots, and prepare test data with predictions.

In [19]:
print(metadata_df.head())
print(metadata_df.index.name)
print(metadata_df.columns)

           latitude  longitude               city            street country
sensor_id                                                                  
60853      62.00000   15.00000      Majorna-Linné        Masthugget  Sweden
59497      62.00000   15.00000          Österskär        Dadelvägen  Sweden
59650      62.00000   15.00000   Tingsryds kommun             G 651  Sweden
112672     57.70716   11.96679         Gothenburg    Bågskyttegatan  Sweden
60889      62.00000   15.00000  Skönsmon District  Björneborgsgatan  Sweden
sensor_id
Index(['latitude', 'longitude', 'city', 'street', 'country'], dtype='object')


In [20]:
# --- Find best model per sensor ---
results_df = pd.DataFrame(results)
best_models = results_df.loc[results_df.groupby("sensor_id")["R2"].idxmax()]

print("Best models per sensor:")
print(best_models[["sensor_id", "feature_name", "R2", "MSE"]])

# --- Cache all feature view data once ---
cached_feature_data = {
    name: fv.query.read()
    for name, fv in feature_views.items()
}

# Normalize dates in cached data
for name, df_cached in cached_feature_data.items():
    df_cached["date"] = pd.to_datetime(df_cached["date"]).dt.tz_localize(None)

all_test_data = []

# --- Loop through best models ---
for _, row in best_models.iterrows():
    sensor_id = row["sensor_id"]
    best_feature = row["feature_name"]

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")

    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()

    # --- EXACT same cleaning as training ---
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "city", "street", "country", "latitude", "longitude", "aqicn_url"]
    ]

    # Keep rows with NaN in features - same as training (XGBoost handles NaN)
    df_clean = df.copy()
    if len(df_clean) < 10:
        continue

    # --- Generate predictions for FULL dataset ---
    X_full = df_clean[feature_cols]
    predictions_full = best_model.predict(X_full)
    df_clean["predicted_pm25"] = predictions_full
    df_clean["best_model"] = best_feature
    
    # --- Filter to last 18 months for hindcast visualization ---
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=18)
    df_hindcast = df_clean[df_clean["date"] >= cutoff_date].copy()
    
    if len(df_hindcast) == 0:
        print(f"⚠️ Skipping sensor {sensor_id}: no data in last 18 months")
        continue

    # Get lat/lon from data (metadata is embedded in feature view)
    if "latitude" in df_clean.columns and "longitude" in df_clean.columns:
        lat = df_clean["latitude"].iloc[0]
        lon = df_clean["longitude"].iloc[0]
    else:
        # Fallback to metadata_df if columns not in feature view
        if sensor_id in metadata_df.index:
            lat = metadata_df.at[sensor_id, "latitude"]
            lon = metadata_df.at[sensor_id, "longitude"]
        else:
            print(f"⚠️ Skipping sensor {sensor_id}: metadata not found")
            continue
        df_clean["latitude"] = lat
        df_clean["longitude"] = lon

    all_test_data.append(
        df_clean[["date", "pm25", "predicted_pm25", "latitude", "longitude", "best_model"]]
    )

Best models per sensor:
     sensor_id feature_name        R2        MSE
362      57421    lagged_1d  0.445156   4.977674
667      58666     complete  0.196094   3.951755
363      58909    lagged_1d  0.347142   4.570608
622      58912     complete  0.463056   5.401411
519      58921    lagged_3d  0.911789   0.047664
..         ...          ...       ...        ...
717     494275     complete  0.699486   0.389391
614     497266    lagged_3d  0.571613   3.341005
615     533086    lagged_3d  0.157948  12.275641
616     556792    lagged_3d  0.219563   2.823703
410     562600    lagged_1d  0.627695   5.850237

[103 rows x 4 columns]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.63s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.43s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.19s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.27s) 
Finished: Reading da

In [21]:
# --- DIAGNOSTIC: Check data coverage by year ---
print("\n📊 DATA COVERAGE DIAGNOSTIC")
print("="*60)

for feature_name, df_cached in cached_feature_data.items():
    print(f"\n{feature_name}:")
    
    # Check one sensor as example
    sample_sensor = df_cached["sensor_id"].iloc[0]
    sensor_data = df_cached[df_cached["sensor_id"] == sample_sensor].copy()
    sensor_data["year"] = pd.to_datetime(sensor_data["date"]).dt.year
    
    print(f"  Sensor {sample_sensor}:")
    print(f"  Total rows: {len(sensor_data)}")
    print(f"  Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
    # Check pm25 by year
    yearly_stats = sensor_data.groupby("year").agg({
        "pm25": ["count", lambda x: x.isna().sum()]
    })
    yearly_stats.columns = ["Total Rows", "NaN Count"]
    yearly_stats["Non-NaN"] = yearly_stats["Total Rows"] - yearly_stats["NaN Count"]
    print("\n  PM2.5 availability by year:")
    print(yearly_stats)
    
    # After dropna
    sensor_data_clean = sensor_data.dropna(subset=["pm25"])
    if len(sensor_data_clean) > 0:
        print(f"\n  After dropna(subset=['pm25']):")
        print(f"    Remaining rows: {len(sensor_data_clean)}")
        print(f"    Date range: {sensor_data_clean['date'].min()} to {sensor_data_clean['date'].max()}")
        print(f"    Years present: {sorted(sensor_data_clean['year'].unique())}")
    else:
        print(f"\n  ⚠️ NO DATA LEFT after dropna!")
    
    break  # Only check first feature view for now

print("\n" + "="*60)


📊 DATA COVERAGE DIAGNOSTIC

baseline:
  Sensor 57421:
  Total rows: 2062
  Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00

  PM2.5 availability by year:
      Total Rows  NaN Count  Non-NaN
year                                
2019          23          0       23
2020         359          0      359
2021         267          0      267
2022         363          0      363
2023         365          0      365
2024         347          0      347
2025         334          0      334
2026           4          0        4

  After dropna(subset=['pm25']):
    Remaining rows: 2062
    Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00
    Years present: [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]



In [22]:
# # Find best model (highest R2) for each sensor
# results_df = pd.DataFrame(results)
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]

# print("Best models per sensor:")
# print(best_models[['sensor_id', 'feature_name', 'R2', 'MSE']])

# all_data = baseline_features.read()
# all_data['date'] = pd.to_datetime(all_data['date']).dt.tz_localize(None)

# cached_feature_data = {
#     name: fv.query.read()
#     for name, fv in feature_views.items()
# }

# all_test_data = []

# for _, row in best_models.iterrows():
#     sensor_id = row['sensor_id']
#     best_feature = row['feature_name']
    
#     sensor_dir = f"{model_dir}/{sensor_id}"
#     if not os.path.exists(sensor_dir):
#         os.mkdir(sensor_dir)
#     images_dir = f"{model_dir}/{sensor_id}/images"
#     if not os.path.exists(images_dir):
#         os.mkdir(images_dir)

#     best_model = models[best_feature][sensor_id]
#     model_path = f"{sensor_dir}/model.json"
#     plot_importance(best_model)
#     importance_path = f"{images_dir}/feature_importance.png"
#     plt.savefig(importance_path)
#     plt.close()
    
#     best_model.save_model(model_path)

#     # Use the same feature view and data processing logic that was used for training
#     best_feature_view = feature_views[best_feature]
#     sensor_data = cached_feature_data[best_feature]
#     sensor_data['date'] = pd.to_datetime(sensor_data['date']).dt.tz_localize(None)
    
#     df = sensor_data[sensor_data['sensor_id'] == sensor_id].copy()
    
#     df = df.sort_values("date").reset_index(drop=True)

#     # Apply EXACT same cleaning logic as in training loop
#     df = df.dropna(subset=['pm25'])
    
#     # Recompute feature columns exactly as in training
#     feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "location_id"]]

#     # Drop rows with NaN in any feature
#     df_clean = df.dropna(subset=feature_cols).copy()

#     if len(df_clean) < 10:
#         continue

#     # Same split as training
#     train_size = int(0.8 * len(df_clean))
#     test_df = df_clean.iloc[train_size:].copy()

#     # # Create feature matrix for comprehensive NaN cleaning (same as training)
#     # features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])
#     # target_for_cleaning = df["pm25"]
    
#     # # Remove rows with NaN values in any feature or target (same as training)
#     # clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#     # df_clean = df[clean_mask].copy()
    
#     # if len(df_clean) < 10:
#     #     continue
        
#     # # Split the cleaned data (same as training)
#     # train_size = int(0.8 * len(df_clean))
#     # test_df = df_clean.iloc[train_size:].copy()
    
#     if len(test_df) == 0:
#         continue
    
#     # Test data is already clean from the comprehensive cleaning above
#     clean_test_df = test_df.copy()
#     predictions = y_preds[best_feature][sensor_id]
    
#     if len(clean_test_df) == len(predictions):
#         clean_test_df['predicted_pm25'] = predictions
#         clean_test_df['best_model'] = best_feature
#         all_test_data.append(clean_test_df[['date', 'pm25', 'predicted_pm25', 'latitude', 'longitude', 'best_model']])
#     else:
#         print(f"⚠️  Skipping sensor {sensor_id}: prediction length mismatch ({len(predictions)} vs {len(clean_test_df)})")

## 3.10. Model Registration & Visualization
Create prediction plots for each sensor, register model in Hopsworks model registry with metrics and metadata, and save models with their configuration.

In [23]:
mr = project.get_model_registry()
df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
df = df.sort_values(by=["date"])

# Plot the best model for each sensor
for sensor_id, meta in metadata_df.iterrows():
    city = meta["city"]
    street = meta["street"]
    latitude = meta["latitude"]
    longitude = meta["longitude"]
    
    df_subset = df[(df["latitude"] == latitude) & (df["longitude"] == longitude)].copy()
    if len(df_subset) == 0:
        continue
    
    # Get the best model name for display
    best_model_name = df_subset['best_model'].iloc[0] if 'best_model' in df_subset.columns else 'unknown'
    best_model_r2 = df_subset['R2'].iloc[0] if 'R2' in df_subset.columns else 0
    best_model_mse = df_subset['MSE'].iloc[0] if 'MSE' in df_subset.columns else 0
    best_model_feature_view = feature_views[best_model_name]
    
    df_subset = df_subset.sort_values(by=["date"])
    df_subset = df_subset.drop(columns=["latitude", "longitude", "best_model"])
    
    images_dir = f"{model_dir}/{sensor_id}/images"
    image_path = f"{images_dir}/hindcast_training.png"
    
    fig = visualization.plot_air_quality_forecast(
        city, street, df_subset, image_path, hindcast=True
    )
    if fig is not None:
        fig.suptitle(f"{city} {street} (Best Model: {best_model_name})")
        plt.close(fig)  # Clean up after saving

    aq_model = mr.python.create_model(
        name=f"air_quality_xgboost_model_{sensor_id}",
        metrics={
            "R2": best_model_r2,
            "MSE": best_model_mse,
        },
        feature_view=best_model_feature_view,
        description=f"Air Quality (PM2.5) predictor for {city} {street} using {best_model_name} configuration",
    )

    aq_model.save(f"{model_dir}/{sensor_id}")

2026-01-15 14:53:24,189 WARNING: ProvenanceWarning: Provenance cached data - feature view provided, but training dataset version is missing

2026-01-15 14:53:24,192 WARNING: ProvenanceWarning: Model schema cannot not be inferred without both the feature view and the training dataset version.



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\krist\Documents\GitHub\pm25/models/60853/model.json: 0.000%|          | 0/515507 elapsed<00…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/60853\images/feature_importance.png: 0.000%|          | …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/60853\images/hindcast_prediction.png: 0.000%|          |…

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/113542\images/feature_importance.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/113542\images/forecast.png: 0.000%|          | 0/46698 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/113542\images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/113542\images/hindcast_training.png: 0.000%|          | …

KeyboardInterrupt: 

## 3.11. Upload visuals to HopsFS

In [ ]:
dataset_api = project.get_dataset_api()

base_dir = "Resources/plots"
try:
    dataset_api.mkdir(base_dir)
except:
    pass

uploaded_images = 0

for sensor_id in metadata_df.index:
    sensor_dir = f"{base_dir}/{sensor_id}"
    try:
        dataset_api.mkdir(sensor_dir)
    except:
        pass

    local_path = f"{model_dir}/{sensor_id}/images/hindcast_training.png"
    remote_path = f"{sensor_dir}/hindcast_training.png" 
    dataset_api.upload( 
        local_path, remote_path, overwrite=True
    )
    uploaded_images = uploaded_images + 1
    print(f"Uploaded image for sensor {sensor_id}, number {uploaded_images} / {len(metadata_df)}")

print(f"Done uploading {uploaded_images} images.")